In [1]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import random
import cv2

In [2]:
def getEdge(img, kernel_size=(5, 5), low_thr=50, high_thr=150):
    # Define a kernel size and apply Gaussian smoothing
    blured = cv2.GaussianBlur(img, kernel_size, 0)

    edges = cv2.Canny(blured, low_thr, high_thr)

    return edges

def getLines(pts_img_like, threshold=1, min_line_length=20, max_line_gap=5):
    # Define the Hough transform parameters
    # Make a blank the same size as our image to draw on
    rho = 1           # pixel resolution: 1 pixel
    theta = np.pi/180 # angle resolution: 1 degree
    
    # Run Hough on edge detected image
    lines = cv2.HoughLinesP(edges, rho, theta, threshold, np.array([]),
                                min_line_length, max_line_gap)
    return lines

def getLineLength(Line):
    return np.sqrt((line[:,0] - line[:,2])**2 + (line[:,1] - line[:,3])**2)

def distanceToLine(line, pt):
    st = line[:,0:2]
    end = line[:,2:]
    vec0 = end - st
    vec1 = pt - st
    area = vec0[:,0]*vec1[:,1] - vec0[:,1]*vec1[:,0]
    length = area / getLineLength(line)
    return length

In [10]:
def calHoughSpace(line, center):
    theta = np.arctan((line[:,3]-line[:,1])/(line[:,2]-line[:,0]))
    r = distanceToLine(line, center)
    return np.stack((theta, r), axis = 1)

def HoughPolarToXYLine(houghPoint, center):
    th_angle = (houghPoint[0]-ScatterResolution/2)/ScatterResolution/ScatterScale
    l = houghPoint[2]/np.sin(th_angle)
    lowerPoint = (int(center[0]+l), center[1])
    upperPoint = (int(center[0]+l-(center[1]-200)/np.tan(th_angle)), 200)
    return lowerPoint, upperPoint

In [11]:
# Read in and grayscale the image
#img = mpimg.imread('exit-ramp.jpg')
cap = cv2.VideoCapture('highway_record4.mp4')

#color=[random.randint(0, 255), random.randint(0, 255),random.randint(0, 255)]
color=[0,255,0]
width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))  # float
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)) # float
fps = cap.get(cv2.CAP_PROP_FPS)
roi_base = (0, height//2)
roi_size = (width, height*3//4)


ScatterScale = 0.5        # Map -1.57~1.57 to 0~resolution
ScatterResolution = 100   # theta * Scale * Resoultion
ScatterDecay = 0.9
houghScatter = np.zeros((ScatterResolution, 3), dtype=np.float)

momLeft = np.array([0, 0, 0, 0])
momRight = np.array([0, 0, 0, 0])
while(cap.isOpened()):
    ret, frame = cap.read()
    
    roi = np.zeros([height,width], dtype=np.uint8)
    roi[height*3//5:height*4//5,0:width] = 255
    
    img = cv2.cvtColor(frame, cv2.COLOR_RGB2HLS)
    roi=cv2.add(img, np.zeros(np.shape(frame), dtype=np.uint8), mask=roi)
    
    lines = []
    #for idx in range(3):
    idx = 1
    channel = roi[:,:,idx]
    #if( idx == 0 ):
    channel = cv2.morphologyEx(channel, cv2.MORPH_OPEN, (3, 3))
    edges = getEdge(channel)
    ll = np.squeeze(getLines(edges, threshold=1, min_line_length=10), axis=1)
    lines.append(ll)
    name = 'edge'+str(idx)
    cv2.imshow(name,roi)
        
    line_image = np.copy(frame) #creating a blank to draw lines on
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(line_image,(x1,y1),(x2,y2),color,2)
    
    for line in lines:
        center = (width//2, 300)
        houghPoints = calHoughSpace(line, center)
        lineLength = getLineLength(line)
        # Filter nan values
        m = np.logical_not(np.isnan(houghPoints[:,1]))
        houghPoints = houghPoints[m]
        lineDisX = abs(line[:,0]+line[:,2])/2 - width/2
        lineDisY = height - abs(line[:,1]+line[:,3])
        lineDis = np.sqrt(lineDisX**2 + lineDisY**2)
        
        # Calculate weight = r * theta
        #factorA = houghPoints[:, 0] * houghPoints[:, 1] / np.mean(houghPoints[:, 1])
        factorB = lineLength / np.mean(lineLength)
        factorC = lineDisY / np.mean(lineDisY)
        factorD = -(lineDis / np.mean(lineDis))
        weight = houghPoints[:, 0] * (factorB*factorC*factorD) * 10
        x = (houghPoints[:, 0] * ScatterScale*ScatterResolution).astype(int)
        
        # Clip into array size
        x = x.clip(-ScatterResolution//2, ScatterResolution//2-1)
        # Weighted add
        houghScatter[:, 1] = 0
        houghScatter[:, 2] = 0
        for (dx, dw, dr) in zip(x, weight, houghPoints[:, 1]):
            houghScatter[dx+ScatterResolution//2, 0] += dw
            houghScatter[dx+ScatterResolution//2, 1] += dr
            houghScatter[dx+ScatterResolution//2, 2] += 1
        houghScatter[houghScatter[:, 2] <= 0.001, 2] = 1000000
        houghScatter[:, 1] /= houghScatter[:, 2]
        
        # Filter minimux & maximum
        houghScatter[0, 0]=0
        houghScatter[-1, 0]=0
        mean = np.mean(houghScatter[:,0])
        scaled = np.column_stack((np.arange(ScatterResolution, dtype=np.int16), 
                                  houghScatter[:,0] / mean, 
                                  houghScatter[:,1:]))
        
        # Calculate outstanding ones
        crtiria = np.logical_and(np.absolute(scaled[:,1])>15, scaled[:,2]>0.001)
        points = scaled[crtiria]
        #print(houghScatter)
        #houghScatter *= ScatterDecay
        houghScatter[:,0:2] *= ScatterDecay
        
        i=0
        for p in houghScatter:
            cv2.line(line_image, (i-50+width//2, 200), (i-50+width//2, int(200-abs(p[0]))), (0, 0, 255), 1)
            cv2.line(line_image, (i-50+width//2, 200), (i-50+width//2, int(200+abs(p[1]))), (0, 255, 0), 1)
            i += 1

        for p in points:
            cv2.circle(line_image, (int(p[0])-50+width//2, 200), 3, (0, 255, 255), 1)
            
        lefts = points[points[:,0]<ScatterResolution//2]
        rights = points[points[:,0]>ScatterResolution//2]
        exist = False
        if(lefts.shape[0] > 0):
            leftMax  = lefts[np.argmax(np.absolute(lefts), axis=0)[1]]
            momLeft = momLeft * 0.9 + leftMax * 0.1
            leftAvr  = np.mean(lefts, axis=0)[0]
            
            cv2.circle(line_image, (int(leftMax[0]) -50+width//2, 200), 5, (255, 255, 0), 2)
            cv2.circle(line_image, (int(leftAvr)-50+width//2, 200) , 5, (255, 0, 0), 2)
            
            (LupperPoint, LlowerPoint) = HoughPolarToXYLine(leftMax, center)
            cv2.line(line_image, LupperPoint, LlowerPoint, (255, 255, 255), 4)
            exist = True
            
        if(rights.shape[0] > 0):
            rightMax = rights[np.argmax(np.absolute(rights), axis=0)[1]]
            momRight = momRight * 0.9 + rightMax * 0.1
            rightAvr = np.mean(rights, axis=0)[0]
            
            cv2.circle(line_image, (int(rightMax[0]) -50+width//2, 200), 5, (255, 255, 0), 2)
            cv2.circle(line_image, (int(rightAvr)-50+width//2, 200) , 5, (255, 0, 0), 2)
            
            (RupperPoint, RlowerPoint) = HoughPolarToXYLine(rightMax, center)
            cv2.line(line_image, RupperPoint, RlowerPoint, (0, 0, 0), 4)
            exist = True
        """
        (upperPoint, lowerPoint) = HoughPolarToXYLine(momLeft, center)
        cv2.line(line_image, upperPoint, lowerPoint, (0, 0, 255), 4)
        (upperPoint, lowerPoint) = HoughPolarToXYLine(momRight, center)
        cv2.line(line_image, upperPoint, lowerPoint, (0, 0, 255), 4)
        """
        if (exist != False):
            rect = np.float32([LupperPoint, RupperPoint, RlowerPoint, LlowerPoint])
            maxWidth = RlowerPoint[0] - LlowerPoint[0]
            maxHeight = RupperPoint[1] - RlowerPoint[1]
            if(maxWidth > 0):
                dst = np.array([
                    [0, 0],
                    [maxWidth - 1, 0],
                    [maxWidth - 1, maxHeight - 1],
                    [0, maxHeight - 1]], dtype = "float32")
                M = cv2.getPerspectiveTransform(rect, dst)
                wrap_image = np.copy(frame)
                wrap_image = cv2.warpPerspective(wrap_image, M, (maxWidth, maxHeight))
                cv2.imshow('name',wrap_image)
    cv2.line(line_image,(width//2,0),(width//2,300),(0, 255, 255),2)
    cv2.imshow('frame',line_image)
    if cv2.waitKey(100) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

E:\anaconda3\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in true_divide
  
